# **Grover's Algorithm for Graph Coloring Problems**
---

## **Introduction** 
This project extends your work in Week 18 to use Grover's algorithm for more advanced problems known as Graph Coloring Problems. You will learn how to build your own an oracle and solve several types of problems that can all be reduced to Graph Coloring.

<br>

## **Description**
In Computer Science, Graph Coloring is one of many so-called "graph problems". Although the mathematical representation and analysis of these problems can get very complex, graph coloring is actually pretty simple. **We can view graph coloring as trying to answer the question, "Can we color every country on a map without any touching countries having the same color, given *k* colors?"**

<br>

It turns out that this problem is extremely powerful and can be used to represent many other problems including:
* Sudoku (numbers instead of colors in a 9X9 map)
* Scheduling
* Networking
* Machine Learning (ex: clustering)

Therefore, solving such a problem is very important!

<br>

We will see how to use Grover's algorithm for this problem, particularly learning how to use Grover to search of a *list of solutions* instead of through a literal databse. This is actually believed to be the more likely use of Grover once we have fault tolerant computers, due to the difficulty of encoding databases into quantum states in part.

We will follow these steps for building and solving a problem with Grover:
1. Create an initial state preparation circuit. This will put all possible solutions in equal superposition.
1. Create an oracle circuit. This will flip the phase of correct solutions and leave the rest alone. Defining this is often the hardest part.
1. Define the problem using Qiskit's `Amplification(...)` function.
1. Solve the problem using Qiskit's `Grover(...)` function. We usually have to run several times with different number of iterations to get a good result.



**NOTE**: This project will involve looking at many histograms of results. As the size of the problems increase, so will the number of solutions we see in our graphs. If you start having trouble seeing the individual bars and states in the graphs, you can click on them to zoom in OR right click and press "Save Image As..." to download onto your computer and open it up in a larger window.

<br>

## **Key Questions**
In this project, you will answer the following questions:
1. When using Grover's algorithm, we do not get good results for too few *or* too many iterations. Rather, there is a "sweet spot" (a balanced number of iterations) right in the middle. Why is this?
2. How and why does having *multiple* correct answers change the number of iterations needed for Grover's algorithm?
3. The best classical algorithm that guarantees solutions to the 4 coloring problem has a Big-O complexity of $O(1.7272^n)$ where $n$ is the number of Countries to be colored (or Routes to be assigned planes). How does Grover's algorithm compare to this run time?

<br>

## **Structure**
This project is broken into 2 parts:
> **Part 1**: Grover for 2 Coloring Problems
>
>> **Part 1.1**: Building a Specific Problem
>>
>> **Part 1.2**: Building General Problems
>>
>> **Part 1.3**: Solving General Problems
>
> **Part 2**: Grover for 4 Coloring Problems
>
>> **Part 2.1**: A First Approach
>>
>> **Part 2.2**: A Faster Approach
>>
>> **Part 2.3**: Beyond Coloring
>
> **Wrapping Up**

<br>

## **Resources**

* [Week 18 Lab Solutions (Grover's Algorithm for Searching Databases)](https://drive.google.com/file/d/1BAYHXCpXLp-iP2WyvamVbRchshwxOquM/view?usp=sharing)
* [Qiskit Textbook: Grover for Sudoku](https://qiskit.org/textbook/ch-algorithms/grover.html#sudoku)
* [Qiskit Documentation: Grover’s Algorithm and Amplitude Amplification](https://qiskit.org/documentation/tutorials/algorithms/06_grover.html)
* [Qiskit Documentation: `AmplificationProblem(...)`](https://qiskit.org/documentation/stubs/qiskit.algorithms.AmplificationProblem.html)
* [Qiskit Documentation: `Grover(...)`](https://qiskit.org/documentation/stubs/qiskit.algorithms.Grover.html)
* [Qiskit Documentation: `circuit.mct(...)`](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.mct.html)

**If you're curious to learn more:**
* [Youtube: The Four Color Theorem - What Counts as a Proof?](https://www.youtube.com/watch?v=42-ws3bkrKM)

---

## **Part 0: Importing from Qiskit**
---

**Run the code below to import everything needed for this lab.**

In [ ]:
!pip install qiskit

from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.circuit import AncillaQubit
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

from qiskit.algorithms import AmplificationProblem
from qiskit.algorithms import Grover

#provider = IBMQ.load_account()

#import math
#from qiskit.quantum_info import Statevector

import warnings
warnings.filterwarnings('ignore')



def plot_results(result, satisfies_disagree_list):

  # Plotting results
  counts = result.circuit_results[0]
  plot = plot_histogram(counts, figsize=(int(6*len(counts.keys())**(1/2)), 8), sort='value_desc')
  ax = plot.gca()
  ax.tick_params(axis='x', which='major', labelsize=10)
  ax.set_xlabel("State", fontsize = 'x-large')
  ax.set_ylabel("Count", fontsize = 'x-large')

  # Marking correct (orange) vs. incorrect (blue) states

  correct_count = 0
  states = ax.xaxis.get_ticklabels()
  for i in range(len(states)):
    if satisfies_disagree_list(states[i].get_text()):
      ax.containers[i][0].set(color = 'orange')
      correct_count += 1
    else:
      ax.containers[i][0].set(color = 'blue')

  leg = plot.legend(['Incorrect (' + str(len(states) - correct_count) + ' counted)', 
                     'Correct (' + str(correct_count) + ' counted)'], 
                    fontsize = 'x-large',
                    loc = 'upper left')
  leg.legendHandles[0].set_color('blue')
  leg.legendHandles[1].set_color('orange')

  display(plot)


print("Libraries Imported Successfully!")

## **Part 1: Grover for 2 Coloring Problems**
---

In this section, we will learn how to use Grover's algorithm to solve the 2 Coloring Problem. In other words, we are trying to color a map with just 2 colors available, which we will choose to be black and white. We will do so through these parts:

**Part 1.1**: Building a Specific Problem

**Part 1.2**: Building General Problems

**Part 1.3**: Solving General Problems

---
### **Part 1.1: Building a Specific Problem**


In this section, we will build and solve a specific 2 coloring problem: two bordering countries with two colors.

**Country 0 | Country 1**

<br>

Therefore, the correct answers are:

**White | Black**

**Black | White**

And the incorrect answers are:

**White | White**

**Black | Black**

<br>

We will use the following representation:

* Each country is represented by a qubit.
* When a qubit is 0, that country is colored white.
* When a qubit is 1, that country is colored black.

This means that the correct solutions correspond to 01 (white and black) and 10 (black and white) where the incorrect solutions correspond to 00 (white and white) and 11 (black and black).


<br>

Lastly, we will follow these steps for building and solving this problem with Grover:
1. Create an initial state preparation circuit. This will put all possible solutions in equal superposition.
1. Create an oracle circuit. This will flip the phase of correct solutions and leave the rest alone. Defining this is often the hardest part.
1. Define the problem using Qiskit's `Amplification(...)` function.
1. Solve the problem using Qiskit's `Grover(...)` function. We usually have to run several times with different number of iterations to get a good result.

<br>

**NOTE**: We will provide you a lot of guidance in this section so that you can focus on implementing the code first. Then, we recommend you go back over this part to really understand it upon a second look.

#### **Step #1: Create an initial state preparation circuit.**

This circuit needs to put all possible solutions in equal superposition. Therefore, complete the code below so that:
* There is 1 qubit per country and no classical bits.
* Each qubit is put into superposition.

In [ ]:
prep = QuantumCircuit(# COMPLETE THIS LINE
                      
prep.# COMPLETE THIS CODE


prep.draw()

#### **Step #2: Create an oracle circuit.**

We need to define a circuit that will flip the phase of correct solutions and leave the rest alone. In this particular case, that means:

* If the two qubits disagree with each other (01 or 10), then we should flip the phase.
* Otherwise (00 or 11), leave the phase alone.

In order to implement this, we will use two extra qubits:
* 1 qubit that says there's a valid solution (01 or 10) by flipping to 1.
* **The "output" bit**: 1 qubit that marks the state by flipping the phase, specifically changing from  $|-\rangle$ to $-|-\rangle$.

This is broken down into 3 parts:
1. Initialize the oracle circuit.
1. Check if the current state is a solution.
1. Reset the extra bit for the next iteration.

##### **1. Initialize the oracle circuit.**

Specifically, create a circuit such that:
* There's 4 qubits and no classical bits.
* The last qubit, $q_3$, is put into the $|-\rangle$ state.

In [ ]:
oracle = # COMPLETE THIS LINE
oracle.# COMPLETE THIS LINE
oracle.# COMPLETE THIS LINE

##### **2. Check if the current state is a solution.**

Specifically, apply gates as follows:
* **Flip $q_2$ if $q_0$ and $q_1$ disagree**: CX controlled by $q_0$ and targeting $q_2$ and a CX controlled by $q_1$ and targeting $q_2$. Technically, this calculates $q_2$ = XOR($q_0$, $q_1$).
* **Flip $q_3$ if $q_2$ has been activated**: CX controlled by $q_2$ and targeting $q_3$.

**NOTE**: Before continuing, it is worth using pen, paper, and kets to understand how these gates accomplish their purposes stated above.

In [ ]:
# Flip qubit 2 if qubits 0 and 1 disagree
oracle.# COMPLETE THIS LINE
oracle.# COMPLETE THIS LINE

# Flip qubit 3 if qubit 2 has been activated
oracle.# COMPLETE THIS LINE

##### **3. Reset the extra bit for the next iteration.**

Since Grover's algorithm often requires multiple uses of the oracle, it is very important that we reset the extra qubit, $q_2$. Otherwise, we may accidentally mark states as valid or not based on what happened in the previous iteration. In practice, this is a matter of just undoing whatever we did to $q_2$.

Specifically, **flip $q_2$ if $q_0$ and $q_1$ disagree**: CX controlled by $q_0$ and targeting $q_2$ and a CX controlled by $q_1$ and targeting $q_2$.

In [ ]:
# Undo flipping qubit 2 if qubits 0 and 1 disagree
oracle.# COMPLETE THIS LINE
oracle.# COMPLETE THIS LINE

oracle.draw()

#### **Step #3: Define the problem using Qiskit's Amplification(...) function.**

Now that we have the state preparation and oracle circuits defined, we can setup the problem for Qiskit. We have provided this code for you, however, it will be valuable to understand this code before trying to solve other problems.

In [ ]:
# Function that takes a string representation of a state
# and checks if the bits disagree with each other
def check_disagreement(state):
  if state[0] == state[1]: return False
  else: return True

# Setting up the problem with Qiskit's AmplificationProblem(...) function
problem = AmplificationProblem(oracle, 
                               state_preparation = prep, 
                               is_good_state = check_disagreement, 
                               objective_qubits = [0, 1])

#### **Step #4: Solve the problem using Qiskit's `Grover(...)` function.**

We have provided you with the code below for 0 iterations, which means we don't check the oracle at all and just prepare an equal superposition of all solutions.

**Run the code, increasing the number of iterations until only correct answers are shown.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Draw the Full Circuit.**

Just for reference, **run the code below to draw the full circuit after you have found the ideal number of iterations.**

**NOTE**: The mysterious looking gates at the beginning, $U3(\pi/2, 0, \pi)$, are actually just another way to write H gates.

In [ ]:
grover.construct_circuit(problem, measurement = True).decompose(reps = 2).draw()

---
### **Part 1.2: Building General Problems**


In this section, we will extend the approach we used below to solve general 2 color problems by placing this code into functions.



**Start by running the code below to define an example problem (the same one as Part 1.1) using variables.**

In [ ]:
variable_qubits = [0, 1]
check_qubits = [2]
output_qubit = 3
disagree_list = [[0,1]]

#### **Exercise #1**

First, let us define a general initial state preparation circuit. Specifically, complete the code below to create a function that:
* Takes in the list of qubit numbers called `variable_qubits`.
* Initializes a quantum circuit with a qubit for every variable and no classical bits.
* Puts all the qubits in `variable_qubits` into equal superpositions.

In [ ]:
def graph_color_prep(variable_qubits):
  
  num_vars = len(variable_qubits)
  prep = # COMPLETE THIS LINE

  prep.h(# COMPLETE THIS CODE

  return prep

#### **Exercise #2**

Now let's work out the code for a general graph coloring oracle part by part. Specifically,

1. Initializing a quantum circuit with the output bit in the $|-\rangle$ state.
1. Checking if each pair of qubits in a given list, `disagree_list`, disagree with each other.
1. Flip the output bit if all disagreements are satisfied.
1. Resetting all the extra qubits for the next iteration.

##### **1. Initializing a quantum circuit with the output bit in the $|-\rangle$ state.**

In [ ]:
num_vars = len(variable_qubits)
num_checks = len(check_qubits)
num_outputs = 1

oracle = QuantumCircuit(# COMPLETE THIS LINE

##### **2. Checking if each pair of qubits in a given list, `disagree_list`, disagree with each other.**

Complete the code below to accomplish this. For reference:
* each element of `disagree_list` is a pair of qubits that we need to disagree with each other. This represents two spots on a map that must be colored differently.
* each element of `check_qubits` is the extra qubit to store the result of checking a pair from `disagree_list`.

**NOTE**: This is a good time to stop and check that you understand what this code is doing before continuing.

In [ ]:
for i in range(len(disagree_list)):
  oracle.cx(disagree_list[i][0], check_qubits[i])
  oracle.cx(# COMPLETE THIS LINE

##### **3. Flip the output bit if all disagreements are satisfied.**

While the CNOT gate worked for the 2 qubit example, we need to generalize this so that there are any number of control qubits. Qiskit provides the multi-controlled target (`mct`) gate for just this purpose.

For example,

* `mct([0], 1) = cx(0, 1)` (CNOT)
* `mct([0, 1], 2) = ccx(0, 1, 2)` (Toffoli)
* `mct([0, 1, 2, 3, 4, 5], 6) = cccccx(0, 1, 2, 3, 4, 5, 6)` (no special name, need to use `mct`)


Use this gate specifically as follows:
* `check_qubits` are the controls.
* `output_qubit` is the target.

In [ ]:
oracle.mct(# COMPLETE THIS LINE

##### **4. Resetting all the extra qubits for the next iteration.**

Lastly, complete the code below to undo everything we did to the extra qubits.

**Hint**: This is undoing the gates applied in 2.

In [ ]:
for i in range(len(disagree_list)):
  oracle.# COMPLETE THIS LINE
  oracle.# COMPLETE THIS LINE

oracle.draw()

#### **Exercise #3**

To truly make a general oracle, let's put this code into a function that we can call later on. Specifically, complete the code below by filling it in with each part of Exercise #2 and returning the oracle circuit.

In [ ]:
def graph_color_oracle(disagree_list, variable_qubits, check_qubits, output_qubit):

  # 1. Initializing a quantum circuit with the output bit in the |−⟩ state.
  # COMPLETE THIS CODE


  # 2. Checking if each pair of qubits in a given list, disagree_list, disagree with each other.
  # COMPLETE THIS CODE

  
  # 3. Flip the output bit if all disagreements are satisfied.
  # COMPLETE THIS CODE


  # 4. Resetting all the extra qubits for the next iteration.
  # COMPLETE THIS CODE

  
  return # COMPLETE THIS LINE

#### **Exercise #4**

Lastly, we will define a useful function for verifying the correct states as follows:
* It takes in a string representation of a measured state, called `state`.
* It takes in `disagree_list`.
* It compares every pair of bits listed in the `disagree_list`.
* If any pair of bits are the same, then the function returns `False`.
* Otherwise, the function returns `True`.

In [ ]:
def check_disagree_list_general(state, disagree_list):
    
    for i in range(len(disagree_list)):
      if state[disagree_list[i][0]] == # COMPLETE THIS LINE
        return False
      
    return # COMPLETE THIS LINE

#### **Exercise #5**

Let's put these functions to use! Specifically,

1. Define the problem.
1. Solve the problem.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1]
check_qubits = [2]
output_qubit = 3
disagree_list = [[0,1]]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(# COMPLETE THIS LINE
oracle = graph_color_oracle(# COMPLETE THIS LINE


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general(state, disagree_list)

problem = AmplificationProblem(oracle,
                               state_preparation = # COMPLETE THIS LINE,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

**Change the number of iterations to the number you found from Part 1.1 and run this code.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

---
### **Part 1.3: Solving General Problems**


So far, we have solved the same problem twice. Let's use the general functions we created above to solve more complicated problems.

#### **Exercise #1**

To start, let's solve a problem with just one more qubit (country to color) such that the countries border each other in a line as follows:

**Country 0 | Country 1 | Country 2**

<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 1 and 2 must be colored differently.
* Countries 0 and 2 can be colored the same or differently since they don't border each other.

And the correct solutions are: 010 and 101.

<br>

Specifically,

1. Define the problem.
1. Solve the problem.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2]
check_qubits = [2, 3]
output_qubit = 4
disagree_list = [[0,1], # COMPLETE THIS LINE TO SPECIFY WHICH PAIRS OF QUBITS MUST DISAGREE


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(# COMPLETE THIS LINE
oracle = graph_color_oracle(# COMPLETE THIS LINE


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general(state, disagree_list)

problem = AmplificationProblem(oracle,
                               state_preparation = # COMPLETE THIS LINE,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

Increase the number of iterations to get the best results. 

**NOTE**: The theoretically optimal number of iterations is roughly $\frac{\pi}{4}\sqrt{\frac{\text{Number of Possible Answers}}{\text{Number of Correct Answers}}}$. As such, for small numbers there often isn't an exact integer number of iterations that will give 100% correct solutions. Instead, the game is to start at something close to this value and tweak it until you get the correct answer most of the time.

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Exercise #2**

Now, solve a problem with 4 countries in the same arrangement:

**Country 0 | Country 1 | Country 2 | Country 3**

<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 1 and 2 must be colored differently.
* Countries 2 and 3 must be colored differently.
* Any other pair of countries can be colored the same or differently since they do not border each other.

And the correct solutions are: 0101 and 1010

<br>

Specifically,

1. Define the problem.
1. Solve the problem.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3]
check_qubits = [# COMPLETE THIS LINE
output_qubit = 7
disagree_list = [[0,1], # COMPLETE THIS LINE TO SPECIFY WHICH PAIRS OF QUBITS MUST DISAGREE


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(# COMPLETE THIS LINE
oracle = graph_color_oracle(# COMPLETE THIS LINE


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general(state, disagree_list)

problem = AmplificationProblem(oracle,
                               state_preparation = # COMPLETE THIS LINE,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

Increase the number of iterations to get the best results.

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Exercise #3**

Now, solve a problem with 4 countries such that Countries 0 and 1 are in a line and Countries 1, 2, and 3 all border each other:

**Country 0 | Country 1 | Country 2**

**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Country 3**

<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 1 and 2 must be colored differently.
* Countries 1 and 3 must be colored differently.
* Countries 2 and 3 must be colored differently.
* Any other pair of countries can be colored the same or differently since they do not border each other.

This problem is particularly interesting since *there are no correct solutions with 2 colors*.

<br>

Specifically,

1. Define the problem.
1. Solve the problem.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3]
check_qubits = [# COMPLETE THIS LINE
output_qubit = 8
disagree_list = [[0,1], # COMPLETE THIS LINE TO SPECIFY WHICH PAIRS OF QUBITS MUST DISAGREE


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(# COMPLETE THIS LINE
oracle = graph_color_oracle(# COMPLETE THIS LINE


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general(state, disagree_list)

problem = AmplificationProblem(oracle,
                               state_preparation = # COMPLETE THIS LINE,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

Increase the number of iterations to get the best results.

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Overconstrained and Underconstrained Problems**

This is an example of an **overconstrained problem** where there are so many constraints that there are actually *no correct answers*. Every algorithm reacts differently to these types of problems. Can you determine what Grover's algorithm is doing using the results above and your understanding of the algorithm? This will be one of the questions to answer in your project summary.

<br>

Technically, all of the problems we have seen so far have been **underconstrained problems** because they had few enough constraints to allow for *multiple correct answers*.


#### **Exercise #4**

To wrap up, solve the following problem involving 6 countries:

**Country 0 | Country 1 | Country 2**

**Country 3 | Country 4 | Country 5**


<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 0 and 3 must be colored differently.
* Countries 1 and 2 must be colored differently.
* Countries 1 and 4 must be colored differently.
* Countries 2 and 5 must be colored differently.
* Countries 3 and 4 must be colored differently.
* Countries 4 and 5 must be colored differently.
* Any other pair of countries can be colored the same or differently since they do not border each other.

Can you determine what the correct answers are (if there are any)?

<br>

Specifically,

1. Define the problem.
1. Solve the problem.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [# COMPLETE THIS LINE
check_qubits = [# COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [# COMPLETE THIS LINE TO SPECIFY WHICH PAIRS OF QUBITS MUST DISAGREE


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(# COMPLETE THIS LINE
oracle = graph_color_oracle(# COMPLETE THIS LINE


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general(state, disagree_list)

problem = AmplificationProblem(oracle,
                               state_preparation = # COMPLETE THIS LINE,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

Increase the number of iterations to get the best results.

**NOTE**: As the size of the problems increase, so will the number of solutions we see in our graphs. If you start having trouble seeing the individual bars and states in the graphs, you can click on them to zoom in OR right click and press "Save Image As...* to download onto your computer and open it up in a larger window.

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

## **Part 2: Grover for 4 Coloring Problems**
---

In this section, we will expand upon the approach we developed in Part 1 to tackle the 4 Coloring Problem. After centuries of debate, it was proven that *any* Graph Coloring problem can be solved with 4 colors. So, being able to use Grover's algorithm to solve the 4 Coloring Problem is enough to solve *any* Graph Coloring problem with Grover's algorithm.

<br>

We will do so through these parts:

**Part 2.1**: A First Approach

**Part 2.2**: A Faster Approach

**Part 2.3**: Beyond Coloring

---
### **Part 2.1: A First Approach**


To start, we will attempt taking a very similar approach to the one we developed above. There are two main adjustments that need to occur:

* To represent 4 possible colors per Country, we need to use *2 qubits*. So now, each Country can be in any of these four states (the colors listed below are just for sake of example): 
  * 00: White
  * 01: Red
  * 10: Orange
  * 11: Black

* To make sure that two Countries are colored differently, we will compare each digit and make sure there is at least one difference. For instance, 00 01 is ok since the 2nd digit (0 and 1) is different in each number even though the first one is the same (0). This first approach will require the use of **ancilla** (extra) qubits.

<br>

We have provided the fully updated graph coloring oracle below. Over the first few exercises, you will define the necessary functions for this to work, `disagree_multi_q` and `undo_disagree_multi_q`.

**Run the code below before moving onto Exercise #1.**

In [ ]:
def graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit):

  # 1. Initializing a quantum circuit with the output bit in the |−⟩ state.
  num_vars = len(variable_qubits)
  num_checks = len(check_qubits)
  num_outputs = 1

  oracle = QuantumCircuit(num_vars + num_checks + num_outputs)
  oracle.x(output_qubit)
  oracle.h(output_qubit)


  # 2. Checking if each pair of qubits in a given list, disagree_list, disagree with each other.
  # Storing the ancilla qubits used for each check
  ancillas = []
  for i in range(len(disagree_list)):
    ancillas += [ disagree_check_4c(oracle,
                                  disagree_list[i][0], 
                                  disagree_list[i][1], 
                                  check_qubits[i]) ]

  
  # 3. Flip the output bit if all disagreements are satisfied.
  oracle.mct(check_qubits, output_qubit)


  # 4. Resetting all the extra qubits for the next iteration.
  # Need to include the specific ancillas used for each check
  for i in range(len(disagree_list)):
    undo_disagree_check_4c(oracle, 
                           disagree_list[i][0], 
                           disagree_list[i][1], 
                           check_qubits[i], 
                           ancillas[i])

  
  return oracle

#### **Exercise #1**

To start, let's work out what the `disagree_check_4c(...)` function needs to do part by part and then we can put it into a function in Exercise #2. The overall goal of this function is to flip the check qubit if at least one pair of digits disagree. We will accomplish this as follows:

1. Add 2 ancilla qubits to the circuit. This provides 1 per digit.
1. Check if the first digit of each state *agrees*.
1. Check if the second digit of each state *agrees*.
1. Flip the check bit if either *disagree*.

**To get started, run the code below. You will refer to these variables throughout this exercise.**

In [ ]:
qc = QuantumCircuit(5)
qubits_a = [0, 1]
qubits_b = [2, 3]
check_qubit = 4

##### **1. Add 2 ancilla qubits to the circuit.**

In [ ]:
ancillas = [AncillaQubit(), # COMPLETE THIS LINE
qc.add_bits(ancillas)

qc.draw()

##### **2. Check if the first digit of each state agrees.**

The following information may be helpful for completing this code:
* The first digit is stored in the 0th index of each: `qubit_a[0]` and `qubit_b[0]`.
* The corresponding ancilla qubit, `ancillas[0]`, should store the result of this check.
* Checking for agreement is the same as checking for disagreement and then flipping the qubit.
* We checked for disagreement of qubits in Part 1 and have generally seen that the X gate can be used to flip qubit states.

In [ ]:
qc.# COMPLETE THIS LINE
qc.# COMPLETE THIS LINE
qc.# COMPLETE THIS LINE

##### **3. Check if the second digit of each state agrees.**

Do the same as above, but for the 2nd digits.

In [ ]:
qc.# COMPLETE THIS LINE
qc.# COMPLETE THIS LINE
qc.# COMPLETE THIS LINE

##### **4. Flip the check bit if either disagree.**

The following information may be helpful for completing this code:
* It's easiest to first check if all the digits agree and then flip the output.
* The multi-controlled target gate allows for checking multiple qubits at once and storing the result.
* The ancillas store the results we need to check.

In [ ]:
qc.# COMPLETE THIS LINE
qc.# COMPLETE THIS LINE

qc.draw()

##### **If you happen to be curious about why we are taking this approach**: 

To allow for one pair of digits to agree and the other to disagree using quantum circuits, we take advantage of the fact that the following two statements are the same:

1. ($q_0$ and $q_2$ disagree) OR ($q_1$ and $q_3$ disagree)

1. NOT( ($q_0$ and $q_2$ agree) AND ($q_1$ and $q_3$ agree) )

The second statement is easier to implement in quantum circuits directly because NOT can be encoded with an X gate and AND can be encoded with the multi-controlled gate (or a Toffoli since we have 2 ancilla qubits as our controls) amongst other approaches. The equivalence of those two statements is technically known as De Morgan's law.

#### **Exercise #2**

Now, complete the code below by filling it in with the parts you developed in Exercise #1. The only addition is that you must also return the ancilla qubits so that this operation can be undone (reset) for future iterations.

In [ ]:
def disagree_check_4c(qc, qubits_a, qubits_b, check_qubit):

  # 1. Add 2 ancilla qubits to the circuit. This provides 1 per digit.
  # COMPLETE THIS CODE


  # 2. Check if the first digit of each state agrees.
  # COMPLETE THIS CODE

  # 3. Check if the second digit of each state agrees.
  # COMPLETE THIS CODE

  # 4. Flip the check bit if either disagree.
  # COMPLETE THIS CODE


  # Return ancillas so that we can undo this operation later on
  return # COMPLETE THIS LINE

#### **Exercise #3**

Next, we need to define the function that *undoes* this operation, `undo_disagree_check_4c(...)`. Instead of returning the ancillas, this function takes them in and uses them to make sure they get reset properly.

**Complete the code below by applying the gates from `disagree_check_4c(...)` in the opposite order.**

In [ ]:
def undo_disagree_check_4c(qc, qubits_a, qubits_b, check_qubit, ancillas):

  # 4. UNDO Flip the check bit if either disagree.
  # COMPLETE THIS CODE

  # 3. UNDO Check if the second digit of each state agrees.
  # COMPLETE THIS CODE

  # 2. UNDO Check if the first digit of each state agrees.
  # COMPLETE THIS CODE

#### **Exercise #4**

Lastly, we need to define an updated version of `check_disagree_list_general(...)` that accounts for the multiple digits: `check_disagree_list_general_4c(...)`.


As a reminder, this function was used for verifying the correct states as follows:
* It takes in a string representation of a measured state, called `state`.
* It takes in `disagree_list`.
* It compares every pair of bits listed in the `disagree_list`.
* If any pair of bits are the same, then the function returns `False`.
* Otherwise, the function returns `True`.

In [ ]:
def check_disagree_list_general_4c(state, disagree_list):

  for i in range(len(disagree_list)):
    
    if (state[disagree_list[i][0][0]] == state[disagree_list[i][1][0]] 
        and state[disagree_list[i][0][1]] == state[disagree_list[i][1][1]]):
        return # COMPLETE THIS LINE

  return # COMPLETE THIS LINE

##### **Run the code below to verify that your function works properly.**

In [ ]:
check1 = (check_disagree_list_general_4c('1111', [[[0, 1], [2, 3]]]) == False)
check2 = (check_disagree_list_general_4c('0111', [[[0, 1], [2, 3]]]) == True)
check3 = (check_disagree_list_general_4c('0110', [[[0, 1], [2, 3]]]) == True)
check4 = (check_disagree_list_general_4c('0101', [[[0, 1], [2, 3]]]) == False)

if not check1:
  print("1111 Fails")

if not check2:
  print("0111 Fails")

if not check3:
  print("0110 Fails")

if not check4:
  print("0101 Fails")

if check1 and check2 and check3 and check4:
  print("Success, keep moving!")

#### **Exercise #5**

Let's put these functions to use! Specifically,

1. Define the problem.
1. Solve the problem.


The problem in this exercise is to color 3 countries that all border each other (something that is impossible to do with just 2 colors):


**Country 0 | Country 1**

**&emsp;&emsp;Country 2**

##### **1. Define the problem.**

Complete the code below to define this specific problem.

**NOTE**: The biggest adjustment here is that every country must be represented by two qubits, which will effect the initial variable defintions.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3, 4, 5]
check_qubits = [6, 7, 8]
output_qubit = 9
disagree_list = [ [[0, 1], [2, 3]],
                  [[0, 1], # COMPLETE THIS LINE,
                  # COMPLETE THIS LINE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Exercise #6**

Now, solve the problem of coloring 4 countries such that the first 2 border each other and the second two border each other, but there is water between them:


**Country 0 | Country 1 ~ ~ water ~ ~ Country 2 | Country 3**


<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 2 and 3 must be colored differently.
* Any other pair of countries can be colored the same or differently since they do not border each other.

<br>

**Question**: Is this problem over- or under- constrained (or is there exactly one solution)?

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3, 4, 5, 6, 7]
check_qubits = [8, # COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ [[0, 1], [2, 3]],
                  # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Exercise #7**

Now, solve the problem of coloring 4 countries such that the first 3 are in the same arrangement as Exercise #5, but are now bordered on all sides by the last country:

**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size="6">Country 3</font>**

**<font size="6">Country 3</font> &ensp;| Country 0 | Country 1 | &ensp;<font size="6">Country 3</font>**

**&ensp;&ensp;&ensp;<font size="6">Country 3</font>&ensp;&ensp;&ensp;| Country 2 | &ensp;&ensp;&ensp;<font size="6">Country 3</font>**

**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size="6">Country 3</font>**


<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 0 and 2 must be colored differently.
* Countries 0 and 3 must be colored differently.
* Countries 1 and 2 must be colored differently.
* Countries 1 and 3 must be colored differently.
* Countries 2 and 3 must be colored differently.
* Any other pair of countries can be colored the same or differently since they do not border each other.

<br>


**NOTE**: Be patient! This is a large problem and may take several minutes to run.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3, 4, 5, 6, 7]
check_qubits = [8, # COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ [[0, 1], [2, 3]],
                  # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Hitting Limits**

The code above should be able to solve this problem within just 1 - 2 iterations. However, it takes almost 2 minutes per iteration. For computers, this is an unacceptably long time, particularly given that this problem is much smaller than many of the applications we might be interested in (most directly, imagine trying to color a map of Africa or the world)!


It is common in scientific research to first find something that works, just so you know it can be done, and then find ways to optimize (improve) it. Furthermore, there are often many ways to optimize an algorithm and choosing how much time and energy to spend on the different ways largely depends on what you need to accomplish in practice.

In Part 2.2, we will explore one way to optimize this code, specifically making it run faster. We can verify that it is faster by trying it out on the exact same problem as above.

---
### **Part 2.2: A Faster Approach**


In this section, we will optimize our implementation of Grover's algorithm for the 4-coloring problem by determining one of the "bottlenecks" (parts that are slowing everything down) and attempting to improve upon it.

#### **Exercise #1**

To start, we need to understand what is actually slowing this code down, called a [bottleneck](https://en.wikipedia.org/wiki/Bottleneck_(software)). In other words, we need to diagnose the problem. 

**Draw the prep and oracle circuits from the problem above to see what we are working with.**

In [ ]:
# DRAW THE PREP CIRCUIT

In [ ]:
# DRAW THE ORACLE CIRCUIT

#### **Open Ended Question**

We cannot do much to change the prep circuit unless we find an entirely different way to represent the problem.

However, there's often more freedom to change the oracle circuit by representing the same logic in a different way. To see what we might change, consider the following question:

***What are different ways we could change the oracle?***

#### **Exercise #2**

The particular way we will optimize the oracle here is to eliminate the need for ancilla qubits as they effectively double the size and complexity of the problem. For instance, the last problem from Part 2.1 had 12 ancilla qubits on top of the 14 "core" qubits.

<br>

To accomplish this, we will approach the disagreement checks completely differently. The logic we used before was very generallly encoding the idea of pairs of qubits disagreeing. However, with 4 colors it turns out that there are only ever 4 cases where all 4 qubits agree with each other. Here are the 4 cases explicitly:

11 11

01 01

10 10

00 00

All (12) other states are acceptable! So, we've been using general logic to eliminate 4/16 or just 1/4 of the possibilities. Instead, let us specifically target these states as follows:

1. Check if the qubits are in the 11 11 state.
2. Check if the qubits are in the 01 01 state.
3. Check if the qubits are in the 10 10 state.
4. Check if the qubits are in the 00 00 state.
5. If none of these states have been found, flip the output qubit.

<br>

**Run the code below to define variables that will be used throughout this exercise.**

In [ ]:
qc = QuantumCircuit(5)
qubits_a = [0, 1]
qubits_b = [2, 3]
check_qubit = 4

##### **1. Check if the qubits are in the 11 11 state.**

To get started, the code for this has been provided for you. Consider why this works to check if $q_0 q_1 q_2 q_3 = 1111$.

In [ ]:
qc.mct(qubits_a + qubits_b, check_qubit)

qc.barrier()
qc.draw()

##### **2. Check if the qubits are in the 01 01 state.**

This is a matter of modifying the approach above by flipping the appropriate qubits AND making sure that you undo any flips *right* after checking so that they are ready to be checked for the next case.

In [ ]:
qc.mct(qubits_a + qubits_b, check_qubit)

qc.barrier()
qc.draw()

##### **3. Check if the qubits are in the 10 10 state.**

In [ ]:
qc.mct(qubits_a + qubits_b, check_qubit)

qc.barrier()
qc.draw()

##### **4. Check if the qubits are in the 00 00 state.**

In [ ]:
qc.mct(qubits_a + qubits_b, check_qubit)

qc.barrier()
qc.draw()

##### **5. If none of these states have been found, flip the output qubit.**

Now all 4 states have been checked and $q_4$ will be 1 if any of them have been found. However, we want to flip this logic so that $q_4$ is 1 if *none* of them have been found. Apply the necessary gate(s) below to accomplish this.

In [ ]:
# COMPLETE THIS CODE

qc.draw()

#### **Exercise #3**

Now, complete the code below by filling it in with the parts you developed in Exercise #2. Unlike in Part 2.1, we do not need to return anything from this function.

<br>

**IMPORTANT NOTE**: Qiskit does not cross barriers when trying to find optimal ways to simulate or compile quantum circuits. So, to make this code as efficient as possible **remove any calls to `qc.barrier()` or `qc.draw()`**.

In [ ]:
def disagree_check_4c(qc, qubits_a, qubits_b, check_qubit):

  # 1. Check if the qubits are in the 11 11 state.
  # COMPLETE THIS CODE

  # 2. Check if the qubits are in the 01 01 state.
  # COMPLETE THIS CODE

  # 3. Check if the qubits are in the 10 10 state.
  # COMPLETE THIS CODE

  # 4. Check if the qubits are in the 00 00 state.
  # COMPLETE THIS CODE

  # 5. If none of these states have been found, flip the output qubit.
  # COMPLETE THIS CODE

#### **Exercise #4**

Lastly, we need to define the function that *undoes* this operation, `undo_disagree_check_4c(...)`.

**Complete the code below by applying the gates from `disagree_check_4c(...)` in the opposite order.**

In [ ]:
def undo_disagree_check_4c(qc, qubits_a, qubits_b, check_qubit):

  # 5. UNDO If none of these states have been found, flip the output qubit.
  # COMPLETE THIS CODE

  # 4. UNDO Check if the qubits are in the 00 00 state.
  # COMPLETE THIS CODE

  # 3. UNDO Check if the qubits are in the 10 10 state.
  # COMPLETE THIS CODE

  # 2. UNDO Check if the qubits are in the 01 01 state.
  # COMPLETE THIS CODE

  # 1. UNDO Check if the qubits are in the 11 11 state.
  # COMPLETE THIS CODE

#### **Updating the Oracle Function**

Now that we are not using ancilla qubits, we need to slightly update the oracle function.

**Run the code below to update the oracle function.**

In [ ]:
def graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit):

  # 1. Initializing a quantum circuit with the output bit in the |−⟩ state.
  num_vars = len(variable_qubits)
  num_checks = len(check_qubits)
  num_outputs = 1

  oracle = QuantumCircuit(num_vars + num_checks + num_outputs)
  oracle.x(output_qubit)
  oracle.h(output_qubit)


  # 2. Checking if each pair of qubits in a given list, disagree_list, disagree with each other.
  for i in range(len(disagree_list)):
    disagree_check_4c(oracle,
                      disagree_list[i][0], 
                      disagree_list[i][1], 
                      check_qubits[i])

  
  # 3. Flip the output bit if all disagreements are satisfied.
  oracle.mct(check_qubits, output_qubit)


  # 4. Resetting all the extra qubits for the next iteration.
  # Need to include the specific ancillas used for each check
  for i in range(len(disagree_list)):
    undo_disagree_check_4c(oracle, 
                           disagree_list[i][0], 
                           disagree_list[i][1], 
                           check_qubits[i])

  
  return oracle

#### **Exercise #5**

Now, solve the problem from the last exercise of Part 2.1.

As a reminder, it is the problem of coloring 4 countries such that the first 3 are in the same arrangement as Exercise #5, but are now bordered on all sides by the last country:

**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size="6">Country 3</font>**

**<font size="6">Country 3</font> &ensp;| Country 0 | Country 1 | &ensp;<font size="6">Country 3</font>**

**&ensp;&ensp;&ensp;<font size="6">Country 3</font>&ensp;&ensp;&ensp;| Country 2 | &ensp;&ensp;&ensp;<font size="6">Country 3</font>**

**&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size="6">Country 3</font>**


<br>

This means that:
* Countries 0 and 1 must be colored differently.
* Countries 0 and 2 must be colored differently.
* Countries 0 and 3 must be colored differently.
* Countries 1 and 2 must be colored differently.
* Countries 1 and 3 must be colored differently.
* Countries 2 and 3 must be colored differently.
* Any other pair of countries can be colored the same or differently since they do not border each other.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3, 4, 5, 6, 7]
check_qubits = [8, # COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ [[0, 1], [2, 3]],
                  # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

---
### **Part 2.3: Beyond Coloring**


Now that we have found a faster way to solve the 4 Coloring problem, let's explore applications of this algorithm beyond the Coloring problem itself. In particular, we will look at **scheduling problems** framed as follows:

> Given a schedule of flying routes and $k$ airplanes that are available to fly, create an assignment of airplanes to flights and timeslots.

<br>


For example the following table shows $k = 4$ planes and 4 total routes to be assigned a plane:

| **Time \| Route** | **Boston to DC (2 hours)** | **DC to Los Angeles (6 hours)** |
|------------------|----------------------------|-------------------------------------|
| **9 am**         | Route 0    | Route 1             |
| **12 pm**        | Route 2    | Route 3             |


<br>

Two possible solutions to this specific problem are:

| **Time \| Route** | **Boston to DC (2 hours)** | **DC to Los Angeles (6 hours)** |
|------------------|----------------------------|-------------------------------------|
| **9 am**         | Airplane 0                 | Airplane 1                          |
| **12 pm**        | Airplane 2                 | Airplane 3                          |


<br>


| **Time \| Route** | **Boston to DC (2 hours)** | **DC to Los Angeles (6 hours)** |
|------------------|----------------------------|-------------------------------------|
| **9 am**         | Airplane 0                 | Airplane 1                          |
| **12 pm**        | Airplane 2                 | Airplane 0                          |



<br>

Most importantly, we can use Graph Coloring to solve this problem! This just requires thinking of the problem as follows:
* Instead of $k$ colors, we have $k$ airplanes.
* Instead of borders, we have restrictions based on the fact that each airplane can only fly one route at a time. In particular, this means:
  * Any airplanes taking off at the same time must be different.
  * An airplane can be used for multiple routes, but only if it has landed after any previous flights. So, the takeoff times and route times must be considered.

#### **Exercise #1**

Let's solve the problem given above and restated here:

| **Time \| Route** | **Boston to DC (2 hours)** | **DC to Los Angeles (6 hours)** |
|------------------|----------------------------|-------------------------------------|
| **9 am**         | Route 0    | Route 1             |
| **12 pm**        | Route 2    | Route 3             |

<br>

Since there are 4 possible planes to use, this is fundamentally a 4 Coloring Problem. So, we can use our Grover solver from above.

The key to doing this is encoding the restrictions correctly:
* Route 0 and Route 1 must use different planes since they take off at the same time.
* Route 0 and Route 2 must use different planes since the plane flying Route 0 will not have time to return to Boston for Route 2.
* Route 1 and Route 2 must use different planes since the plane flying Route 1 will still be in the air when Route 2 is meant to start.
* Route 1 and Route 3 must use different planes for the same reason.
* Route 2 and 3 must use different planes since they take off at the same time.
* The only acceptable reuse of a plane is for Route 0 and 3 since the plane flying Route 0 will have been able to land in DC before Route 3 takes off.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [0, 1, 2, 3, 4, 5, 6, 7]
check_qubits = [8, # COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ [[0, 1], [2, 3]],
                  # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Exercise #2**

Now, let's solve a bigger problem:

| **Time \| Route** | **Boston to DC (2 hours)** | **DC to Los Angeles (6 hours)** |
|-------------------|----------------------------|---------------------------------|
| **9 am**          | Route 0                    | Route 1                         |
| **12 pm**         | Route 2                    | Route 3                         |
| **3 pm**          | Route 4                    | Route 5                         |

<br>

The restrictions are the same as above, but now include:
* Route 1 and Route 4 must use different planes since the plane flying Route 1 will have just landed in Los Angeles and so cannot takeoff from Boston.
* Route 1 and Route 5 must use different planes for the same reason.
* Route 2 and Route 3 must use different planes since they take off at the same time.
* Route 2 and Route 4 must use different planes since the plane flying Route 2 will not have time to return to Boston for Route 4.
* Route 3 and Route 4 must use different planes since the plane flying Route 3 will still be in the air when Route 4 is meant to start.
* Route 3 and Route 5 must use different planes for the same reason.
* Route 4 and 5 must use different planes since they take off at the same time.

<br>

**NOTE**: The size of the problem and the fact there's multiple correct solutions means this problem may take a long time to run and, even more so, to solve well. As such it is ok if you do not get perfect results for this problem. Rather, we leave it to you to continue on after completing the project notebook to determine a way to get around these problems. This could include:
* Using a more efficient simulator.
* Considering if any variables can be removed from the problem. For instance, a Route that cannot overlap with *any* other Route can be assigned a plane and then both that Route and plane can be completely removed from the problem (meaning it is now a 3 Coloring Problem).
* Redefining the way we use Grover's algorithm.

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [# COMPLETE THIS LINE
check_qubits = [# COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

#### **Exercise #3**

Now, apply what you have learned so far to solving this problem:

| **Time \| Route** | **Boston to DC (2 hours)** | **DC to Los Angeles (6 hours)** | **DC to Boston (2 hours)** |
|-------------------|----------------------------|---------------------------------|----------------------------|
| **9 am**          | Route 0                    | Route 1                         | Route 2                    |
| **12 pm**         | Route 3                    | Route 4                         | Route 5                    |

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [# COMPLETE THIS LINE
check_qubits = [# COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

##### **1. Define the problem.**

Complete the code below to define this specific problem.

In [ ]:
# DEFINE THE VARIABLES
variable_qubits = [# COMPLETE THIS LINE
check_qubits = [# COMPLETE THIS LINE
output_qubit = # COMPLETE THIS LINE
disagree_list = [ # COMPLETE THIS CODE
                 ]


# CREATE THE PREP AND ORACLE CIRCUITS
prep = graph_color_prep(variable_qubits)
oracle = graph_color_oracle_4c(disagree_list, variable_qubits, check_qubits, output_qubit)


# DEFINE THE AmplificationProblem
def check_disagreement(state): return check_disagree_list_general_4c(state, disagree_list)

problem = AmplificationProblem(oracle, 
                               state_preparation = prep,
                               objective_qubits = variable_qubits,
                               is_good_state = check_disagreement
                               )

##### **2. Solve the problem.**

In [ ]:
grover = Grover(iterations = 0, quantum_instance = Aer.get_backend('qasm_simulator'))
results = grover.amplify(problem)

plot_results(results, check_disagreement)

## **Wrapping Up**

*Congratulations!* You have built and used a way to solve the 4 Coloring Graph Problem using Grover's algorithm. This means you can technically now use Grover's algorithm to solve *all* Graph Coloring Problems! Furthermore, you have seen how to turn a very different sounding problem into a 4 coloring problem so that you can use this same algorithm.

To wrap up, please make sure to write a summary of the concepts covered here, including answers to the following questions:

1. When using Grover's algorithm, we do not get good results for too few *or* too many iterations. Rather, there is a "sweet spot" (a balanced number of iterations) right in the middle. Why is this?
2. How and why does having *multiple* correct answers change the number of iterations needed for Grover's algorithm?
3. The best classical algorithm that guarantees solutions to the 4 coloring problem has a Big-O complexity of $O(1.7272^n)$ where $n$ is the number of Countries to be colored (or Routes to be assigned planes). How does Grover's algorithm compare to this run time?

# End of Lab

---

© 2023 The Coding School, All rights reserved